In [14]:
%reload_ext autoreload
%autoreload 2

import os


from hydra import compose, initialize, core
from omegaconf import OmegaConf

from src.custom_utils import S3Helper
from src.opt import optimize_vehicle_allocation

## Hydra

In [15]:
core.global_hydra.GlobalHydra.instance().clear()
initialize(version_base="1.2", config_path="../src/config", job_name="optimization")
config = OmegaConf.to_container(compose(config_name="main"), resolve=True)

## Short-Term Forecasts

In [16]:
s3_helper = S3Helper()

short_term_forecasts = s3_helper.read_parquet(
    obj_key=os.path.join(
        config["short"]["forecasts_key"], "short_term_avg_forecasts.parquet"
    )
)

short_term_forecasts

,bus,rail_boardings
2023-07-01,328739.109592,288233.043486
2023-07-02,243357.984727,216357.703007
2023-07-03,447245.738188,337283.620625
2023-07-04,511491.640493,410591.781609
2023-07-05,511364.574161,414590.829922
2023-07-06,502382.244099,423152.064936
2023-07-07,468112.814418,387144.620487
2023-07-08,326816.630180,289649.314392
2023-07-09,242113.805427,218156.392891
2023-07-10,445683.714400,338186.659429


## Linear (Integer) Programming

### Objective Function

Minimize the total operating costs over \( n \) days:

$$
\text{Minimize } Z = \sum_{i=1}^{n} \left( c_{\text{bus40}} \cdot b_{i,40} + c_{\text{bus60}} \cdot b_{i,60} + c_{\text{rail}} \cdot r_i \right)
$$

where:
- $c_{\text{bus40}}$ is the operating cost per day for a 40-foot bus
- $c_{\text{bus60}}$ is the operating cost per day for a 60-foot bus
- $c_{\text{rail}}$ is the operating cost per day for a railcar
- $b_{i,40}$ is the number of 40-foot buses used on day $i$
- $b_{i,60}$ is the number of 60-foot buses used on day $i$
- $r_i$ is the number of railcars used on day $i$
- $n$ is the total number of days in the planning horizon

### Capacity Constraints 

Ensure that the number of vehicles allocated meets or exceeds the forecasted demand for each day, considering the number of trips each vehicle can make.

For buses:
$$
(b_{i,40} \cdot k_{\text{bus40}} + b_{i,60} \cdot k_{\text{bus60}}) \cdot \text{trips per bus} \geq d_{\text{bus},i} \quad \forall i \in \{1, 2, \ldots, n\}
$$

For railcars:

$$
r_i \cdot k_{\text{rail}} \cdot \text{trips per rail} \geq d_{\text{rail},i} \quad \forall i \in \{1, 2, \ldots, n\}
$$

### Additional Constraints

Ensure that at least a certain percentage of buses are 40-foot buses:
$$
b_{i,40} \geq 0.10 \cdot (b_{i,40} + b_{i,60}) \quad \forall i \in \{1, 2, \ldots, n\}
$$

### Non-Negative and Integer Constraints

The number of 40-foot buses, 60-foot buses, and railcars must be non-negative integers:

$$
b_{i,40}, b_{i,60}, r_i \geq 0 \quad \text{and integers} \quad \forall i \in \{1, 2, \ldots, n\}
$$

In [17]:
# Constants for two types of buses
bus_40_capacity = 53
bus_60_capacity = 80
operating_cost_per_bus_40 = 150
operating_cost_per_bus_60 = 180

# Constants for railcars
rail_capacity = 80
operating_cost_per_railcar = 300

# Number of trips per bus and railcar
trips_per_bus = 5
trips_per_rail = 3

# Percentage of 40ft buses in the final allocation
bus_40_percentage = 0.8

days = short_term_forecasts.index

results = optimize_vehicle_allocation(
    forecast=short_term_forecasts,
    cost_bus_40=operating_cost_per_bus_40,
    cost_bus_60=operating_cost_per_bus_60,
    cost_rail=operating_cost_per_railcar,
    capacity_bus_40=bus_40_capacity,
    capacity_bus_60=bus_60_capacity,
    capacity_rail=rail_capacity,
    trips_per_bus=trips_per_bus,
    trips_per_rail=trips_per_rail,
    percentage_40_foot=bus_40_percentage,
)

results

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kenwu/opt/anaconda3/envs/ts_env/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/z_/pqnd1zt14yb6fdthpr75md480000gn/T/223f1325d06546f2a172a4af86439e5f-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/z_/pqnd1zt14yb6fdthpr75md480000gn/T/223f1325d06546f2a172a4af86439e5f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 47 COLUMNS
At line 244 RHS
At line 287 BOUNDS
At line 330 ENDATA
Problem MODEL has 42 rows, 42 columns and 70 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 9.43173e+06 - 0.00 seconds
Cgl0003I 0 fixed, 28 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 28 rows, 28 columns (28 integer (0 of which binary)) and 56 elements
Cutoff increment increased from 1e-05 to 29.9999
Cbc0012I Integer solu

,Date,Optimal Number of 40-foot Buses,Optimal Number of 60-foot Buses,Optimal Number of Railcars
0,2023-07-01,901.0,225.0,1201.0
1,2023-07-02,668.0,166.0,902.0
2,2023-07-03,1226.0,306.0,1406.0
3,2023-07-04,1402.0,350.0,1711.0
4,2023-07-05,1403.0,349.0,1728.0
5,2023-07-06,1377.0,344.0,1764.0
6,2023-07-07,1285.0,319.0,1614.0
7,2023-07-08,897.0,223.0,1207.0
8,2023-07-09,665.0,165.0,909.0
9,2023-07-10,1223.0,304.0,1410.0
